## Convolutional Neural Network

This notebook demonstrates a convolutional neural network that predicts the whether an Ising Model lattice was simulated below or above the Curie Temperature.

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as r
import MetroSim as met
import tensorflow as tf
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as l
import tensorflow.keras.utils as u

In [38]:
# Preparing the training and test data
# Must be normalized
TJ = 2/np.log(1+np.sqrt(2)) # The known phase change temperature

def pol(x): # Function to immitate a Heaveside step
    if x>TJ:
        x = 1
    else: 
        x = 0
    return x

data = pd.read_csv('16_25000_2800.csv') # Download a premade data set
# randomizing the data with the temperature index
randomized = np.random.permutation(data)
r= [i[1:] for i in randomized] 
name = [pol(i[0]) for i in randomized]


train_images = np.array([i.reshape(16,16,1) for i in r[:60000]])
train_labels = np.array([i for i in name[:60000]])
test_images = np.array([i.reshape(16,16,1) for i in r[60001:]])
test_labels = np.array([i for i in name[60001:]])

train_labels = u.to_categorical(train_labels)
test_labels = u.to_categorical(test_labels)


In [42]:
print(np.shape(train_images))
print(np.shape(train_labels))

(60000, 16, 16, 1)
(60000, 2)


In [36]:
# Defining the Model
model = Sequential()
model.add(l.Conv2D(64,(2,2), activation='relu', input_shape=(16, 16,1)))
model.add(l.Dense(64, activation='relu'))
model.add(l.Dropout(0.5))
model.add(l.Flatten())
model.add(l.Dense(2, activation='softmax'))
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 15, 15, 64)        320       
_________________________________________________________________
dense_13 (Dense)             (None, 15, 15, 64)        4160      
_________________________________________________________________
dropout_7 (Dropout)          (None, 15, 15, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 14400)             0         
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 28802     
Total params: 33,282
Trainable params: 33,282
Non-trainable params: 0
_________________________________________________________________


In [39]:
# Training
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.fit(train_images, train_labels,
          batch_size=100,
          epochs=5,
          verbose=3)

Train on 60000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5


In [41]:
# Testing
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=3)
print('Test accuracy: ', test_acc)

Test accuracy:  0.95466477
